# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 20 2022 2:18PM,249095,10,MSP215079,"Methapharm, Inc.",Released
1,Oct 20 2022 2:18PM,249095,10,MSP215080,"Methapharm, Inc.",Released
2,Oct 20 2022 2:18PM,249095,10,MSP215083,"Methapharm, Inc.",Released
3,Oct 20 2022 2:18PM,249095,10,MSP215086,"Methapharm, Inc.",Released
4,Oct 20 2022 2:15PM,249094,10,0086092505,ISDIN Corporation,Released
5,Oct 20 2022 2:15PM,249094,10,0086092508,ISDIN Corporation,Released
6,Oct 20 2022 2:15PM,249094,10,0086092506,ISDIN Corporation,Released
7,Oct 20 2022 2:15PM,249094,10,0086092507,ISDIN Corporation,Released
8,Oct 20 2022 2:15PM,249094,10,0086092515,ISDIN Corporation,Released
9,Oct 20 2022 2:15PM,249094,10,0086092521,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,249091,Released,1
21,249092,Released,1
22,249093,Released,5
23,249094,Released,11
24,249095,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249091,NaN,NaN,1.0
249092,NaN,NaN,1.0
249093,NaN,NaN,5.0
249094,NaN,NaN,11.0
249095,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248896,0.0,1.0,0.0
249002,0.0,1.0,1.0
249007,7.0,1.0,1.0
249068,0.0,2.0,8.0
249070,8.0,23.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248896,0,1,0
249002,0,1,1
249007,7,1,1
249068,0,2,8
249070,8,23,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248896,0,1,0
1,249002,0,1,1
2,249007,7,1,1
3,249068,0,2,8
4,249070,8,23,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248896,,1,
1,249002,,1,1
2,249007,7,1,1
3,249068,,2,8
4,249070,8,23,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 20 2022 2:18PM,249095,10,"Methapharm, Inc."
4,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation
15,Oct 20 2022 2:13PM,249093,10,"Methapharm, Inc."
18,Oct 20 2022 2:13PM,249093,10,Methapharm-G
20,Oct 20 2022 2:09PM,249092,10,Yusen – 3D Matrix
21,Oct 20 2022 2:04PM,249090,10,Bio-PRF
29,Oct 20 2022 2:04PM,249091,10,Emerginnova
30,Oct 20 2022 1:38PM,249088,21,"NBTY Global, Inc."
31,Oct 20 2022 1:05PM,249086,16,ACG North America LLC
32,Oct 20 2022 12:28PM,249082,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 20 2022 2:18PM,249095,10,"Methapharm, Inc.",,,4
1,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation,,,11
2,Oct 20 2022 2:13PM,249093,10,"Methapharm, Inc.",,,5
3,Oct 20 2022 2:13PM,249093,10,Methapharm-G,,,5
4,Oct 20 2022 2:09PM,249092,10,Yusen – 3D Matrix,,,1
5,Oct 20 2022 2:04PM,249090,10,Bio-PRF,,,8
6,Oct 20 2022 2:04PM,249091,10,Emerginnova,,,1
7,Oct 20 2022 1:38PM,249088,21,"NBTY Global, Inc.",,,1
8,Oct 20 2022 1:05PM,249086,16,ACG North America LLC,,,1
9,Oct 20 2022 12:28PM,249082,10,ISDIN Corporation,,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 2:18PM,249095,10,"Methapharm, Inc.",4,,
1,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation,11,,
2,Oct 20 2022 2:13PM,249093,10,"Methapharm, Inc.",5,,
3,Oct 20 2022 2:13PM,249093,10,Methapharm-G,5,,
4,Oct 20 2022 2:09PM,249092,10,Yusen – 3D Matrix,1,,
5,Oct 20 2022 2:04PM,249090,10,Bio-PRF,8,,
6,Oct 20 2022 2:04PM,249091,10,Emerginnova,1,,
7,Oct 20 2022 1:38PM,249088,21,"NBTY Global, Inc.",1,,
8,Oct 20 2022 1:05PM,249086,16,ACG North America LLC,1,,
9,Oct 20 2022 12:28PM,249082,10,ISDIN Corporation,14,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 2:18PM,249095,10,"Methapharm, Inc.",4,,
1,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation,11,,
2,Oct 20 2022 2:13PM,249093,10,"Methapharm, Inc.",5,,
3,Oct 20 2022 2:13PM,249093,10,Methapharm-G,5,,
4,Oct 20 2022 2:09PM,249092,10,Yusen – 3D Matrix,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 2:18PM,249095,10,"Methapharm, Inc.",4.0,NaN,NaN
1,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation,11.0,NaN,NaN
2,Oct 20 2022 2:13PM,249093,10,"Methapharm, Inc.",5.0,NaN,NaN
3,Oct 20 2022 2:13PM,249093,10,Methapharm-G,5.0,NaN,NaN
4,Oct 20 2022 2:09PM,249092,10,Yusen – 3D Matrix,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 2:18PM,249095,10,"Methapharm, Inc.",4.0,0.0,0.0
1,Oct 20 2022 2:15PM,249094,10,ISDIN Corporation,11.0,0.0,0.0
2,Oct 20 2022 2:13PM,249093,10,"Methapharm, Inc.",5.0,0.0,0.0
3,Oct 20 2022 2:13PM,249093,10,Methapharm-G,5.0,0.0,0.0
4,Oct 20 2022 2:09PM,249092,10,Yusen – 3D Matrix,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2988884,60.0,4.0,14.0
15,249070,11.0,23.0,8.0
16,249086,1.0,0.0,0.0
19,249002,1.0,1.0,0.0
21,497984,1.0,1.0,0.0
22,996310,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2988884,60.0,4.0,14.0
1,15,249070,11.0,23.0,8.0
2,16,249086,1.0,0.0,0.0
3,19,249002,1.0,1.0,0.0
4,21,497984,1.0,1.0,0.0
5,22,996310,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,60.0,4.0,14.0
1,15,11.0,23.0,8.0
2,16,1.0,0.0,0.0
3,19,1.0,1.0,0.0
4,21,1.0,1.0,0.0
5,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,60.0
1,15,Released,11.0
2,16,Released,1.0
3,19,Released,1.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Completed,14.0,8.0,0.0,0.0,0.0,0.0
Executing,4.0,23.0,0.0,1.0,1.0,0.0
Released,60.0,11.0,1.0,1.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Completed,14.0,8.0,0.0,0.0,0.0,0.0
1,Executing,4.0,23.0,0.0,1.0,1.0,0.0
2,Released,60.0,11.0,1.0,1.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Completed,14.0,8.0,0.0,0.0,0.0,0.0
1,Executing,4.0,23.0,0.0,1.0,1.0,0.0
2,Released,60.0,11.0,1.0,1.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()